In [23]:
import requests
import json
import time

In [28]:
url = "https://gateway.thegraph.com/api/d9f53ff70c6b397d4fa9cc6e0f17f6d7/subgraphs/id/7nuSuPhUgKSg5uKRh8g5jyjZWE8DnvQw1mQtvDWpxrnh"

def retrieve_proposals():
    body = f"""
    {{
        proposals(
            first: 1000,
            orderBy: creationTime,
            orderDirection: desc
        ) {{
            id
            creationTime
            abstainDelegateVotes
            abstainWeightedVotes
            againstDelegateVotes
            againstWeightedVotes
            executionTime
            description
            forDelegateVotes
            forWeightedVotes
            proposer {{
              id
            }}
            quorumVotes
            state
            totalDelegateVotes
            totalWeightedVotes
        }}
    }}
    """
    
    response = requests.post(url=url, json={"query": body})
    
    if response.status_code == 200:
        return response.json()
    else:
        return []

In [29]:
with open ('compound_proposals.json', 'w', encoding="utf8") as outfile:
    json.dump(retrieve_proposals(), outfile, ensure_ascii=False)

In [30]:
with open('compound_proposals.json', 'r', encoding="utf8") as file:
    data = json.load(file)

In [34]:
url = "https://gateway.thegraph.com/api/d9f53ff70c6b397d4fa9cc6e0f17f6d7/subgraphs/id/7nuSuPhUgKSg5uKRh8g5jyjZWE8DnvQw1mQtvDWpxrnh"

def retrieve_votes(proposal):
    body = f"""
    {{
        votes(
            where: {{
                proposal: "{proposal}",
            }},
            orderBy: weight,
            orderDirection: desc
        ) {{
            voter{{
                id
            }}
            choice
            blockTimeId
            weight
        }}
    }}
    """
    
    response = requests.post(url=url, json={"query": body})
    
    if response.status_code == 200:
        return response.json().get("data", {}).get("votes", [])
    else:
        return []

In [ ]:
all_votes = []

for proposal in data['data']['proposals']:
    proposal_id = proposal['id']
    votes = retrieve_votes(proposal_id)
    
    all_votes.append({
        "proposal_id": proposal_id,
        "votes": votes
    })
    
with open ('compound_votes.json', 'w', encoding="utf8") as outfile:
    json.dump(all_votes, outfile, ensure_ascii=False)